<a href="https://colab.research.google.com/github/juliojesus15/SankeyDiagramMapping/blob/main/MappingData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
print (":D")

:D
